In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
combined_df = pd.read_csv('./big_data_ignore/april_may_combined.csv', low_memory=False)

In [3]:
#clean_april = pd.read_csv('./big_data_ignore/clean_april.csv', low_memory=False)
#clean_may = pd.read_csv('./big_data_ignore/clean_may.csv', low_memory=False)

In [4]:
complaint_ix = pd.read_csv('./big_data_ignore/2016-06-03-complaint-index.csv', encoding='latin-1')
media_ix = pd.read_csv('./big_data_ignore/2016-06-03-all-media-files-index.csv', encoding='latin-1')                

complaint_ix.rename(columns={'COMPLAINT_NUMBER': 'complaint_number'}, inplace=True)
media_ix.rename(columns={'COMPLAINT_NUMBER': 'complaint_number'}, inplace=True)

In [5]:
full = pd.merge(combined_df, complaint_ix, on='complaint_number', how='right')
full = pd.merge(full, media_ix, on='complaint_number', how='left')

In [6]:
full['accused_full_name'] = full.accused_officer_fname.astype('str') + '_' + \
                            full.accused_officer_lname.astype('str')
full['involved_full_name'] = full.involved_officer_fname.astype('str') + '_' + \
                             full.involved_officer_lname.astype('str')
full['ipra_investigator_full_name'] = full.ipra_investigator_firstname.astype('str') + '_' + \
                                      full.ipra_investigator_lastname.astype('str')

In [7]:
full = full.where(pd.notnull(full), None)
full = full.applymap(lambda x: None if x=='nan_nan' or x=='novalue' else x)

In [10]:
cols = [col for col in full.columns if col != 'complaint_number']
group = full.groupby('complaint_number')
df_list = [group[col].unique().reset_index() for col in cols]

In [35]:
summary_df = df_list[0]
for col_df in df_list[1:]:
    summary_df = pd.merge(summary_df, col_df, on='complaint_number')

In [36]:
def get_names(nlist, fname=True):
    if fname:
        return [name.split('_')[0] for name in nlist if name is not None]
    else:
        return [name.split('_')[1] for name in nlist if name is not None]

summary_df.accused_officer_fname = summary_df.accused_full_name.apply(get_names)
summary_df.accused_officer_lname = summary_df.accused_full_name.apply(get_names, fname=False)
summary_df.involved_officer_fname = summary_df.involved_full_name.apply(get_names)
summary_df.involved_officer_lname = summary_df.involved_full_name.apply(get_names, fname=False)
summary_df.ipra_investigator_firstname = summary_df.ipra_investigator_full_name.apply(get_names)
summary_df.ipra_investigator_lastname = summary_df.ipra_investigator_full_name.apply(get_names, fname=False)

In [37]:
def clear_lists(x):
    if len(x) == 1: 
        return x[0]
    else:
        no_none = [val for val in x 
                   if val is not None and str(val)!='nan' and str(val)!='NaN']
        if len(no_none) == 0:
            return None
        elif len(no_none) == 1:
            return no_none[0]
        else:
            return no_none
        
cols = summary_df.columns.tolist()[1:]       
summary_df[cols] = summary_df[cols].applymap(clear_lists)

In [46]:
summary_df.to_csv('./summary_ipra.csv', index=False)